In [574]:
import time

import numpy as np
import pandas as pd
import selenium
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import (NoSuchElementException,
                                        StaleElementReferenceException,
                                        TimeoutException)
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [501]:
driver = webdriver.Chrome()

In [502]:
driver.get('https://www.linkedin.com')

In [503]:
with open('./login_credential.txt') as f:
    lines = f.read().splitlines()

In [504]:
username = driver.find_element(By.ID, 'session_key')
password = driver.find_element(By.ID, 'session_password')

In [505]:
username.send_keys(lines[0])

time.sleep(5)
password.send_keys(lines[1])

In [506]:
driver.find_element(By.CLASS_NAME, 'sign-in-form__submit-button').click()

In [507]:
driver.get('https://linkedin.com/jobs')
time.sleep(10)

In [508]:
job_search = driver.find_element(By.XPATH, '//*[@id="jobs-search-box-keyword-id-ember25"]')
job_search.send_keys('data')

time.sleep(2)

In [509]:
location_search = driver.find_element(By.XPATH, '//*[@id="jobs-search-box-location-id-ember25"]')
location_search.send_keys('United Kingdom')

time.sleep(2)

In [510]:
job_search.send_keys(Keys.RETURN)
time.sleep(5)

In [511]:
def get_job_elems(driver):
    return driver.find_elements(By.XPATH, '//*[@id="main"]/div/section[1]/div/ul/*')

job_list = driver.find_elements(By.XPATH, '//*[@id="main"]/div/section[1]/div/ul/*')

In [512]:
driver.refresh()

In [513]:

job_list = driver.find_elements(By.XPATH, '//*[@id="main"]/div/section[1]/div/ul/*')

In [514]:
len(job_list)

0

In [515]:
import re

def get_id(driver):
    link_elem = driver.find_element(By.XPATH, 
                        '//div[contains(@class, "jobs-unified-top-card t-14")]//a[@href]')
    link = link_elem.get_attribute('href')
    _id = re.findall('(?<=view/)(.*)(?=/)', link)

    assert len(_id) == 1, "Multiple Id found for same job posting!"

    return int(_id[0]) 

In [768]:
def get_skills(driver):
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, 
                        "//button[contains(@aria-label, 'View strong skill match modal')]"))
        )
        try:
            driver.find_element(By.XPATH, 
                            "//button[contains(@aria-label, 'View strong skill match modal')]").click()
        except StaleElementReferenceException:
            driver.find_element(By.XPATH, 
                            "//button[contains(@aria-label, 'View strong skill match modal')]").click()
    except TimeoutException:
        return False

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, 
                        "//ul[contains(@class, 'job-details-skill-match-status-list')]"))
        )
    except TimeoutException:
        return

    skill = driver.find_elements(By.XPATH, 
                            "//ul[contains(@class, 'job-details-skill-match-status-list')]")

    text = skill[0].text.replace("Add", '')

    driver.find_element(By.XPATH, "//div[contains(@aria-labelledby, 'jobs-skill-match-modal-header')]//button").click()

    return [skill for skill in text.split("\n") if skill != '']
    
    

In [746]:
def generate_skill_df(elem):
    action = ActionChains(driver)
    action.move_to_element(elem).perform()
    elem.click()
    _id = get_id(elem)
    _skills = get_skills(driver)

    if _skills == False:
        return 

    _df = pd.DataFrame({'id': [_id for x in range(len(_skills))], \
                        'skills': _skills})

    return(_df)

In [747]:
def generate_all_skill_df(driver, elems):
    df_list = []
    for elem in elems:
        _df = generate_skill_df(elem)
        df_list.append(_df)
    
    df_list = [df for df in df_list if ~isinstance(df, type(None))]
    return pd.concat(df_list)

In [520]:
job_list = get_job_elems(driver)
# test_all = generate_all_skill_df(driver, job_list)

In [521]:
test_all

,id,skills
0,3476935826,Machine Learning
1,3476935826,Computer Science
2,3476935826,Curriculum
3,3476935826,Cyber-security
4,3476935826,Data Mining
...,...,...
5,3479438501,Databases
6,3479438501,Microsoft Power BI
7,3479438501,Operations Management
8,3479438501,Programming Languages


In [522]:
get_skills(driver)

False

In [564]:
soup = bs(driver.page_source)
test = soup.find_all(class_=re.compile('jobs-unified-top-card__job-insight'))
tuple([y for y in [x.find(string=re.compile('.*employees.*')) for x in test] if y][0].split('·') )

('51-200 employees ', ' Staffing and Recruiting')

In [524]:
def scrape_job(driver):
    soup = bs(driver.page_source)

In [761]:
def set_execute(func):
    func.execute = True
    return func

class linkedin_soup(object):

    def __init__(self, soup, auto_execute = True):
        self.soup = soup
        if auto_execute:
            self.auto_generate()
    
    @set_execute
    def get_link(self):
        self.link = 'https://www.linkedin.com' + self.soup.find(class_=re.compile('jobs-unified-top-card__content.*')).\
                                                        find('a')['href']

    @set_execute
    def get_id(self):
        if 'link' not in self.__dict__.keys():
            self.get_link()
        _id = re.findall('(?<=view/)(.*)(?=/)', self.link)

        assert len(_id) == 1, 'Multiple id detected'
        self.id = int(_id[0])


    @set_execute
    def get_title(self):
        title_elem = self.soup.find_all(class_='t-24 t-bold jobs-unified-top-card__job-title')
        assert len(title_elem) == 1, "More than 1 job title found on page!"

        title = title_elem[0].contents[0]
        assert isinstance(title, str)

        self.title = title

    @set_execute
    def get_company(self):
        try:
            self.company =  self.soup.find(class_ = 'jobs-unified-top-card__company-name').text.strip()
        except AttributeError:
            self.company_name =  np.nan

    @set_execute
    def get_location(self):
        try:
            self.location =  self.soup.find(class_="jobs-unified-top-card__bullet").text.strip()
        except AttributeError:
            self.location =  np.nan

    @set_execute
    def get_job_type(self):
        try:
            self.job_type =  self.soup.find(class_=re.compile(".*workplace-type")).text
        except AttributeError:
            self.job_type =  np.nan
        
    @set_execute
    def get_employees_company(self):
        insight = self.soup.find_all(class_=re.compile('jobs-unified-top-card__job-insight'))
        emp_comp = [string for string in [match.find(string=re.compile(".*employees.*")) 
                                for match in insight] if string][0]
        try:
            self.employee, self.company_type = tuple([data.strip() for data in emp_comp.split("·")])
        except ValueError:
            self.employee = emp_comp.strip()
            self.company_type = np.nan

    @set_execute
    def get_salary(self):
        try:
            self.salary = self.soup.find(href='#SALARY').text
        except AttributeError:
            self.salary = np.nan

    @set_execute
    def get_job_desc(self):
        self.job_desc = self.soup.select("div#job-details > span")[0].get_text(separator="\n").strip()
    
    def auto_generate(self):
        for method in dir(self):
            if getattr(getattr(self, method), 'execute', False):
                getattr(self, method)()
    
    def generate_df(self):
        return pd.DataFrame({key:item for key, item in self.__dict__.items() if key != 'soup'}, 
            index = [0])

        


In [728]:
soup = bs(driver.page_source)
test = linkedin_soup(soup)

In [729]:
test.generate_df()

,company,employee,company_type,link,job_desc,job_type,location,salary,title
3479453724,identifi Global Resources,11-50 employees,Staffing and Recruiting,https://www.linkedin.com/jobs/view/3479453724/...,TECHNICAL BUSINESS ANALYS\nT - no sponsorship ...,Hybrid,"Lancashire, England, United Kingdom","£50,000/yr",Technical Business Analyst


In [720]:
pd.DataFrame({key:item for key, item in test.__dict__.items() if (key != 'soup') & (key != 'id')}, 
            index = [test.__dict__['id']])

,company,employee,link,job_desc,job_type,location,salary,title
3479453724,Staffing and Recruiting,11-50 employees,https://www.linkedin.com/jobs/view/3479453724/...,\n\nTECHNICAL BUSINESS ANALYS\nT - no sponsors...,Hybrid,"Lancashire, England, United Kingdom","£50,000/yr",Technical Business Analyst


In [694]:
test_link = soup.find(class_=re.compile('jobs-unified-top-card__content.*')).find('a')['href']


In [696]:
import webbrowser

webbrowser.open('https://www.linkedin.com' + test_link)

True

In [659]:
test.auto_generate()

In [ ]:
test.__di

In [645]:
test.get_company()
test.get_location()

In [607]:
def wrapper(func):
    func.is_test = True
    return func

@wrapper
def blah(x):
    return

blah(1)

blah.what_the = True


In [528]:
class listing_wait(object):

    def __init__(self, locator, xpath):
        self.locator = locator
        self.xpath = xpath

    def __call__(self, driver):
        _elements = driver.find_elements(self.locator, self.xpath)
        if len(_elements) == 0:
            return False
        else:
            return _elements

In [541]:

# driver.refresh()
# # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, 
# #                         "//ul[contains(@class, 'scaffold-layout__list-container')]")))

# time.sleep(15)

# job_elems = get_job_elems(driver)
# print(len(job_elems))
# action = ActionChains(driver)

# for elem in job_elems:
#     action.move_to_element(elem).perform()
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable(elem))
#     elem.click()
#     # test = WebDriverWait(driver, 10).until(listing_wait(By.XPATH, 
#     #                     "//h2[contains(@class, 't-24 t-bold jobs-unified-top-card__job-title')]"))
#     WebDriverWait(driver, 10, ignored_exceptions=StaleElementReferenceException).until(EC.element_to_be_clickable((By.XPATH, 
#                                 "//div[contains(@class, 'jobs-apply-button')]//button")))
#     # time.sleep(1)
#     soup = bs(driver.page_source)
#     print(get_title(soup), get_company(soup), get_location(soup))

25
Business Systems Analyst Systematix Reading, England, United Kingdom
Data Science Consultant  Women in Data® London, England, United Kingdom
Data Analytics & AI Consultant Women in Data® London, England, United Kingdom
Data Analyst more2 London, England, United Kingdom
Data Analyst MSX International Warwick, England, United Kingdom
Aspire Data Analyst Landsec London, England, United Kingdom
Technical Business Analyst identifi Global Resources Lancashire, England, United Kingdom
Data Management Consultant Cloud Decisions United Kingdom
Machine Learning Engineer (Jnr/Mid/Snr) - World Renowned Music Streaming Platform bluAI London, England, United Kingdom
Data Strategists (All levels) Atkins London, England, United Kingdom
Data Analyst Adecco London, England, United Kingdom
Analytics Consultant acceleration London, England, United Kingdom
Data Strategy Consultant  Women in Data® London, England, United Kingdom
Programmes Data Analyst Gallagher London, England, United Kingdom
Data Analy

soup = bs(driver.page_source)

In [769]:
def scrape_page(driver):
    driver.refresh()
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, 
    #                         "//ul[contains(@class, 'scaffold-layout__list-container')]")))

    time.sleep(15)

    job_elems = get_job_elems(driver)
    assert len(job_elems) == 25, "Job listing less than 25, web page not properly loaded!"

    skill_dfs = []
    job_dfs = []
    for elem in job_elems:
        action = ActionChains(driver)
        action.move_to_element(elem).perform()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(elem))
        elem.click()
        # test = WebDriverWait(driver, 10).until(listing_wait(By.XPATH, 
        #                     "//h2[contains(@class, 't-24 t-bold jobs-unified-top-card__job-title')]"))
        try:
            WebDriverWait(driver, 10, ignored_exceptions=StaleElementReferenceException).until(EC.element_to_be_clickable((By.XPATH, 
                                        "//div[contains(@class, 'jobs-apply-button')]//button")))
        except TimeoutException:
            WebDriverWait(driver, 10, ignored_exceptions=StaleElementReferenceException).until(EC.presence_of_element_located((By.XPATH, 
                                        "//div[contains(@class, 'feedback--success')]")))
        
        skill_dfs.append(generate_skill_df(elem))
        _soup = bs(driver.page_source)
        _linkedin_soup = linkedin_soup(_soup)
        job_dfs.append(_linkedin_soup.generate_df())

    return (pd.concat(skill_dfs, ignore_index=True), pd.concat(job_dfs, ignore_index=True))



In [753]:
skill_df, job_df = scrape_page(driver)

In [759]:
skill_df

,id,skills
0,3485104391,Python (Programming Language)
1,3485104391,SQL
2,3485104391,Azure Data Factory
3,3485104391,Azure DevOps
4,3485104391,Data Lakes
...,...,...
0,3485055768,Data Science
1,3485055768,Machine Learning
2,3485055768,Python (Programming Language)
3,3485055768,Modeling


In [755]:
job_df

,company,employee,company_type,link,job_desc,job_type,location,salary,title
0,BCN Group,201-500 employees,IT Services and IT Consulting,https://www.linkedin.com/jobs/view/3485104391/...,Job title:\n \nSolutions Developer (Data Engin...,Remote,United Kingdom,"£40,000/yr - £50,000/yr",Data Engineer
1,Billigence,5,NaN,https://www.linkedin.com/jobs/view/3478254736/...,Billigence has been providing business intelli...,Remote,United Kingdom,NaN,Data Lead (Capital Markets/Trader Industry)
2,The Pensions Regulator,"501-1,000 employees",Government Administration,https://www.linkedin.com/jobs/view/3479263216/...,Role Title:\n \nAnalytics and Data Science Int...,Hybrid,"Brighton and Hove, England, United Kingdom","£18,965/yr",Analytics and Data Science Internship
3,Hays,"5,001-10,000 employees",Staffing and Recruiting,https://www.linkedin.com/jobs/view/3469053010/...,Join an established international biotech. Pro...,Hybrid,"Slough, England, United Kingdom",NaN,Data Scientist
4,Women in Data®,1-10 employees,Information Services,https://www.linkedin.com/jobs/view/3482721044/...,Apply here:\n \nwww.womenindata.co.uk/job/data...,Hybrid,"London, England, United Kingdom",NaN,Data Science Consultant
5,Women in Data®,1-10 employees,Information Services,https://www.linkedin.com/jobs/view/3482718180/...,Apply here:\n \nwww.womenindata.co.uk/job/data...,Hybrid,"London, England, United Kingdom",NaN,Data Analytics & AI Consultant
6,QinetiQ,"5,001-10,000 employees",Defense and Space Manufacturing,https://www.linkedin.com/jobs/view/3482678532/...,It’s an exciting time to join one of our Early...,NaN,"Farnborough, England, United Kingdom",NaN,Data Engineering/Date Science Graduate - Early...
7,Brox.AI,1,NaN,https://www.linkedin.com/jobs/view/3468501162/...,Brox.AI is a funded tech company that is revol...,Remote,United Kingdom,"$60,000/yr - $100,000/yr",Market Research Analyst
8,Slimming World,5,NaN,https://www.linkedin.com/jobs/view/3474863736/...,Slimming World is the UK’s leading weight mana...,Hybrid,"Alfreton, England, United Kingdom",NaN,Digital Insights Analyst
9,Measurelab,11-50 employees,Marketing Services,https://www.linkedin.com/jobs/view/3477219791/...,Job description\nSalary\n: £30k - £45k\nLocati...,Remote,United Kingdom,NaN,Analytics Engineer


In [770]:
def scrape_pages(driver, page_num):
    skill_dfs = []
    job_dfs = []

    for page in range(1, page_num + 1):
        print(f"Page {page}")
        page_button = driver.find_element(By.XPATH, f"//button[contains(@aria-label, 'Page {page}')]")
        action = ActionChains(driver)
        action.move_to_element(page_button)
        page_button.click()

        time.sleep(15)

        _skill_df, _job_df = scrape_page(driver)

        skill_dfs.append(_skill_df)
        job_dfs.append(_job_df)

    return (pd.concat(skill_dfs, ignore_index=True), pd.concat(job_dfs, ignore_index=True))

In [771]:
scrape_pages(driver, 40)

Page 1


TypeError: object of type 'NoneType' has no len()